In [ ]:
import pybullet as p
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 10000 #ms
URDF = "./resource/kuka_iiwa.urdf"

def get_rot(T):
    R = e.Matrix3d()
    for row in range(R.rows()):
        for col in range(R.cols()):
            R.coeff(row, col, T.coeff(row, col))
    return R

def get_pos(T):
    pos = e.Vector3d()
    for i in range(pos.rows()):
        pos[i] = T.coeff(i,3)
    return pos

def get_col(R, c=0):
    assert c <= R.cols(), "column index out of bounds"
    assert R.rows() == R.cols(), "square matrix expected"
    assert R.rows() == 3, "3x3 matrix expected"
    v = e.Vector3d()
    for i in range(R.rows()):
        v[i] = R.coeff(i,c)
    return v

def get_orientation_error(R, R_d):
    '''Calculates orientation error
    @param R Current orientation in Mat3d
    @param R_d Desired orientation in Mat3d
    @return del_phi "R_d-R" error
    '''
    c1 = get_col(R,0)
    c2 = get_col(R,1)
    c3 = get_col(R,2)
    d1 = get_col(R_d,0)
    d2 = get_col(R_d,1)
    d3 = get_col(R_d,2)
    
    return 1.0/2.0*(c1.cross(d1)+c2.cross(d2)+c3.cross(d3))
    
    
def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    pos = s.translation();
    
    for row in range(R.rows()):
        for col in range(R.cols()):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(pos.rows()):
        m4d.coeff(row, 3, pos[row]);
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))
    
# import robot as global variable
r = from_urdf_file(URDF)
print("Imported " + r.name.decode("utf-8"))

# set gravity direction (this is the acceleration at base joint for RNEA)
r.mbc.gravity = e.Vector3d(0,0,9.81)
r.mbc.zero(r.mb)

# setup physics
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
p.setGravity(0,0,-9.81)
p.setRealTimeSimulation(0)
p.setTimeStep(DT)

# import robot
plane_id = p.loadURDF("plane.urdf")
robot_id = p.loadURDF(URDF, [0,0,0], p.getQuaternionFromEuler([0,0,0]), 
                     flags=(p.URDF_USE_INERTIA_FROM_FILE|p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS))

# parse number of joints except fixed joint
print("Auto parse revolute and fixed joints only")
joints_id = []
for i in range(p.getNumJoints(robot_id)):
    if(p.getJointInfo(robot_id, i)[2] == p.JOINT_REVOLUTE):
        joints_id.append(i)
joints_id = np.array(joints_id)
n_dof = len(joints_id)

# gains
Kp = e.MatrixXd(np.diag([10000,10000,10000,40000,40000,40000]))
Kd = e.MatrixXd(np.diag([200,200,200,400,400,400]))

# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S.coeff(0,3,1)
S.coeff(1,4,1)
S.coeff(2,5,1) # linear
S.coeff(3,0,1)
S.coeff(4,1,1)
S.coeff(5,2,1) # angular

# task definitions
body_name_1 = "lbr_iiwa_link_7"
S1 = S

# setup configuration
p.setJointMotorControlArray(robot_id, joints_id, p.VELOCITY_CONTROL, forces=[0] * n_dof)
q_home = np.zeros(n_dof)
for i in joints_id:
    p.resetJointState(robot_id, i, targetValue=q_home[i])
    p.enableJointForceTorqueSensor(robot_id, i)
q_home = e.VectorXd(q_home)


# shows the link frame defined by <inertial><origin> in urdf, -1 is the first link
com_ee = list(p.getDynamicsInfo(robot_id, n_dof-1)[3]) # last link
start = [-x for x in com_ee]
end_x = start.copy()
end_y = start.copy()
end_z = start.copy()
end_x[0] += 0.3
end_y[1] += 0.3
end_z[2] += 0.3
p.addUserDebugLine(start, end_x, lineColorRGB=[1, 0, 0], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=6)
p.addUserDebugLine(start, end_y, lineColorRGB=[0, 1, 0], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=6)
p.addUserDebugLine(start, end_z, lineColorRGB=[0, 0, 1], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=6)
 
# setup camera
p.resetDebugVisualizerCamera(5,0,0,[0,0,1.18])

def set_q_in_mbc(q):
    for i in range(0, n_dof):
        r.mbc.q[i+1][0] = q[i]

def set_dq_in_mbc(dq):
    for i in range(0, n_dof):
        r.mbc.alpha[i+1][0] = dq[i]
    
# gains
Kp_joint = e.MatrixXd(np.diag([8000]*n_dof))
Kd_joint = e.MatrixXd(np.diag([180]*n_dof))
Ko_joint = e.MatrixXd(np.diag([100]*n_dof))
    
# observer
residual = e.VectorXd([0] * n_dof)
residual_int = e.VectorXd([0] * n_dof)
    
# simulate torque control
for i in range(10000):
    joint_states = p.getJointStates(robot_id, joints_id)
    
    # update q and dq states from simulation
    q_bullet = [joint_states[i][0] for i in joints_id]
    dq_bullet = [joint_states[i][1] for i in joints_id]

    # update states into mbc
    set_q_in_mbc(q_bullet)
    set_dq_in_mbc(dq_bullet)
    
    # updates states into "eigen" representation
    q = e.VectorXd(q_bullet)
    dq = e.VectorXd(dq_bullet)

    ############################### forward kinematics to update model ###############################
    rbd.forwardKinematics(r.mb, r.mbc)
    rbd.forwardVelocity(r.mb, r.mbc)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(0,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * r.mbc.bodyPosW[body_id_from_name(body_name_1, r.mb.bodies())])
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(r.mb, body_name_1.encode('utf-8'), ofsPos)
    J1_temp = e.MatrixXd(6, r.mb.nrDof());
    dJ1_temp = e.MatrixXd(6, r.mb.nrDof());
    jac.fullJacobian(r.mb, jac.jacobian(r.mb, r.mbc), J1_temp);
    jac.fullJacobian(r.mb, jac.jacobianDot(r.mb, r.mbc), dJ1_temp);
    J1 = S*J1_temp
    dJ1 = S*dJ1_temp
    
    ############################### forward dynamics to update model ###############################
    fd = rbd.ForwardDynamics(r.mb)
    
    # mass matrix & nonlinear effects 
    fd.computeH(r.mb, r.mbc)
    M = fd.H()
    fd.computeC(r.mb, r.mbc)
    h = fd.C()
    
    # coriolis matrix
    cor = rbd.Coriolis(r.mb)
    C = cor.coriolis(r.mb, r.mbc)
    
    # gravity
    set_dq_in_mbc([0] * n_dof)
    rbd.forwardKinematics(r.mb, r.mbc)
    fd = rbd.ForwardDynamics(r.mb)
    fd.computeC(r.mb, r.mbc)
    g = fd.C()
    set_dq_in_mbc(dq_bullet)
    rbd.forwardKinematics(r.mb, r.mbc)
    rbd.forwardVelocity(r.mb, r.mbc)
    
    # apply force
    p.applyExternalForce(robot_id, joints_id[-1], (0,0,10), (0,0,0), p.LINK_FRAME)

    # control
    err = e.VectorXd([0,np.pi/4,0,-np.pi/2,0,np.pi/4,0])-q
    derr = -dq
    tau = e.VectorXd(M*(Kp_joint*err+Kd_joint*derr)) + h

    # momentum observer
    beta = g - C.transpose()*dq
    residual_int += (residual+tau+C.transpose()*dq-g)*DT
    residual = Ko_joint*(M*dq-residual_int)
    Jc = e.MatrixXd(J1)
    JcT = Jc.transpose()
    JcT_inv = e.MatrixXd(np.linalg.pinv(JcT))
    f = JcT_inv*residual
    
    # printing
    if i % 100 == 0:
        print("{}\n".format(f.transpose()))
    
    # command system
    for i in joints_id:
        p.setJointMotorControl2(robot_id, i, p.TORQUE_CONTROL, force=tau[i])
        
    # step simulation
    p.stepSimulation()
#     time.sleep(DT)
    
time.sleep(1)
p.disconnect()

pybullet build time: Mar 26 2022 03:00:52


Imported lbr_iiwa
Auto parse revolute and fixed joints only
     -6432.8  4.89162e+15 -2.13836e+15  3.07634e+15      1250.92     -659.345

  -5.02851 -0.0699747    1.32209 -0.0720595  -0.807212 -0.0113437

 -0.0290695 -0.00645623    -10.0367 0.000478733  -0.0272519  0.00107367



In [ ]:
# import pybullet as p
# import time
# import pybullet_data

# cid = p.connect(p.SHARED_MEMORY)
# if (cid < 0):
#     print("GUI")
#     p.connect(p.GUI)

# p.setAdditionalSearchPath(pybullet_data.getDataPath())  
# p.loadURDF("plane.urdf")
# kuka = p.loadURDF("kuka_iiwa/model.urdf")
# p.addUserDebugText("tip", [0, 0, 0.1],
#                    textColorRGB=[1, 0, 0],
#                    textSize=1.5,
#                    parentObjectUniqueId=kuka,
#                    parentLinkIndex=6)
# p.addUserDebugLine([0, 0, 0], [0.1, 0, 0], [1, 0, 0], parentObjectUniqueId=kuka, parentLinkIndex=6)
# p.addUserDebugLine([0, 0, 0], [0, 0.1, 0], [0, 1, 0], parentObjectUniqueId=kuka, parentLinkIndex=6)
# p.addUserDebugLine([0, 0, 0], [0, 0, 0.1], [0, 0, 1], parentObjectUniqueId=kuka, parentLinkIndex=6)
# p.setRealTimeSimulation(0)
# angle = 0
# while (True):
#   time.sleep(0.01)
#   p.resetJointState(kuka, 2, angle)
#   p.resetJointState(kuka, 3, angle)
#   angle += 0.01